In [ ]:
# pip install 'farm-haystack[all]'

In [ ]:
# !pip install datasets
# !pip install sacremoses

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from haystack.nodes import DensePassageRetriever
from datasets import load_dataset
import pandas as pd
import numpy as np

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


In [ ]:
dataset = load_dataset("clarin-knext/fiqa-pl", 'corpus')
dataset=pd.DataFrame(dataset['corpus'])
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating corpus split:   0%|          | 0/57638 [00:00<?, ? examples/s]

,_id,title,text
0,3,,"Nie mówię, że nie podoba mi się też pomysł szk..."
1,31,,Tak więc nic nie zapobiega fałszywym ocenom po...
2,56,,Nigdy nie możesz korzystać z FSA dla indywidua...
3,59,,Samsung stworzył LCD i inne technologie płaski...
4,63,,Oto wymagania SEC: Federalne przepisy dotycząc...
...,...,...,...
57633,599946,,">Cóż, po pierwsze, drogi to coś więcej niż hob..."
57634,599953,,"Tak, robią. Na dotacje dla firm farmaceutyczny..."
57635,599966,,">To bardzo smutne, że nie rozumiesz ludzkiej n..."
57636,599975,,„Czy Twój CTO pozwolił dużej grupie użyć „„adm...


In [ ]:
dataset3 = load_dataset("clarin-knext/fiqa-pl", "queries")
df_queries = pd.DataFrame(dataset3['queries'])
df_queries.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating queries split:   0%|          | 0/6648 [00:00<?, ? examples/s]

,_id,title,text
0,0,,Co jest uważane za wydatek służbowy w podróży ...
1,4,,Wydatki służbowe - ubezpieczenie samochodu pod...
2,5,,Rozpoczęcie nowego biznesu online
3,6,,„Dzień roboczy” i „termin płatności” rachunków
4,7,,Nowy właściciel firmy – Jak działają podatki d...


In [ ]:
dataset2 = load_dataset("clarin-knext/fiqa-pl-qrels")
df_qa_test = pd.DataFrame(dataset2['test'])
df_qa_test.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

,query-id,corpus-id,score
0,8,566392,1
1,8,65404,1
2,15,325273,1
3,18,88124,1
4,26,285255,1


## Configure a document store based on Faiss supported by multilingual E5 model:


In [ ]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", similarity="dot_product")
document_store.save(index_path="data/my_index.faiss", config_path="data/my_config.json")

In [ ]:
dicts = []
for id, text in zip(dataset['_id'], dataset['text']):
  dicts.append({
        'content': "</s>"+text,
        'meta': {'id': id}
    })

document_store.write_documents(dicts)

In [ ]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="ipipan/silver-retriever-base-v1",
    passage_embedding_model="ipipan/silver-retriever-base-v1"
)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Auto-detected model language: english


In [1]:
document_store.update_embeddings(retriever)

In [ ]:
text = 'Pytanie: ' + df_queries['text'][0]

In [ ]:
all_documents = document_store.get_all_documents()
print("All Documents:")
print(all_documents[:5])

All Documents:
[<Document: {'content': '</s>Zacznę robić jedzenie, które ludzie kochają. 1517 jest stary. Jedzenie też powinno być stare. Dobry rynek do wprowadzania nowych odmian żywności. Zaletą jest to, że mogę eksperymentować, co chcę; swobodnie robić wszystko. Za każdym razem, gdy wprowadzam na rynek nowe danie, istnieje bardzo duże prawdopodobieństwo, że ludzie zwrócą szczególną uwagę na nowe smaki. Pieniądze nie byłyby tam głównym punktem zwrotnym. Karmienie ludzi różnymi potrawami przyniosłoby reputację społeczności. To wszystko, czego chcę, aby moja firma rozwijała się płynnie.', 'content_type': 'text', 'score': None, 'meta': {'vector_id': '0', 'id': '91305'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '1001d52f8467f3f008c58d1fcdd0e358'}>, <Document: {'content': '</s>To pochodna długu. Swapy na zwłokę w spłacie kredytu to kontrakty zawarte na konkretną emisję obligacji (w przypadku obligacji jednopodmiotowych) lub na koszyk/indeks klasy obligacji (w przypadku swapów

In [ ]:
from haystack.pipelines import DocumentSearchPipeline

pipe = DocumentSearchPipeline(retriever)

In [ ]:
prediction = pipe.run(
    query=text, params={"Retriever": {"top_k": 5}}
)

In [ ]:
print(prediction['documents'][0].meta['id'])

65404


In [ ]:
from haystack.utils import print_documents

print_documents(prediction, max_text_len=200)



Query: Pytanie: Co jest uważane za wydatek służbowy w podróży służbowej?

{   'content': '</s>Czy rozumiesz, dlaczego prywatne firmy płacą za '
               'podróżowanie służbową lub pierwszą klasą swoich pracowników? '
               'To dlatego, że klasa trenera jest do bani. Zabierze cię tam '
               'klasa trenera, zmęczona pod...',
    'name': None}

{   'content': '</s>W porządku, IRS Publikacja 463: Podróże, rozrywka, '
               'prezenty i wydatki na samochód Biznes i użytek osobisty. Jeśli '
               'korzystasz z samochodu zarówno do celów służbowych, jak i '
               'osobistych, musisz podzi...',
    'name': None}

{   'content': '</s>Chris, ponieważ jesteś właścicielem własnej firmy, nikt '
               'nie może powstrzymać Cię od obciążania Twojego konta firmowego '
               'wydatkami osobistymi. IRS nie jest wielkim fanem mieszania '
               'wydatków służbowyc...',
    'name': None}

{   'content': '</s>Większość prywatnych

## Use the set of questions and the scorings defined in this corpus, to compute NDCG@5 for the dense retriever.


In [ ]:
def calc_ndcg_k(true, predicted):
    dcg = np.sum(predicted / np.log2(np.arange(2, len(true) + 2)))
    idcg = np.sum(true / np.log2(np.arange(2, len(true) + 2)))
    ndcg = dcg / idcg
    return ndcg

In [ ]:
max_matches = df_qa_test.groupby('query-id')['corpus-id'].count().rename('count')
max_matches

query-id
8        2
15       1
18       1
26       2
34       1
        ..
10979    4
10994    2
11039    8
11054    2
11088    1
Name: count, Length: 648, dtype: int64

In [ ]:
arr = np.array([0.0 for i in range(5)])
ndcg = 0
i = 0

for query_id in df_qa_test['query-id'].unique():
    query = str('Pytanie: ' + df_queries[df_queries['_id'] == str(query_id)].iloc[0]['text'])
    prediction = pipe.run(
    query=query, params={"Retriever": {"top_k": 5}}
    )
    corpus_ids = df_qa_test[df_qa_test['query-id'] == query_id]['corpus-id']

    tmp = set()
    for ind in corpus_ids:
        tmp.add(ind)

    matches = min(max_matches.loc[query_id], 5)
    true = np.array([0.0 for i in range(5)])
    true[:matches] = 1
    for ind, val in enumerate(prediction['documents']):
        _id = int(val.meta['id'])
        if _id in tmp:
            arr[ind] = 1
        else:
            arr[ind] = 0
    ndcg += calc_ndcg_k(true, arr)
    i += 1

In [ ]:
mean_ndcg_silver = ndcg / i
mean_ndcg_silver = round(mean_ndcg_silver, 3)

print(f"Średnia NDCG wynosi {mean_ndcg_silver}")

Średnia NDCG wynosi 0.232


## Combine dense retrieval with classification model from lab 6 to implement a two-step retrieval. Compute NDCG@5 for this combined model.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = 'gdrive/MyDrive/NLP'


In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=path + "/result/checkpoint-2400")

In [ ]:
arr = np.array([0.0 for i in range(5)])

ndcg = 0
i = 0

for query_id in df_qa_test['query-id'].unique():
    query = str('Pytanie: ' + df_queries[df_queries['_id'] == str(query_id)].iloc[0]['text'])
    prediction = pipe.run(
    query=query, params={"Retriever": {"top_k": 10}}
    )
    corpus_ids = df_qa_test[df_qa_test['query-id'] == query_id]['corpus-id']

    data = []
    for val in prediction['documents']:
      data.append(f"{query} Kontekst: {val}")

    tmp = set()
    for ind in corpus_ids:
        tmp.add(int(ind))

    matches = min(max_matches.loc[query_id], 5)
    true = np.array([0.0 for i in range(5)])
    true[:matches] = 1
    rank = {}
    for j, text in enumerate(data):
      text = text[:512]
      rank[j] = classifier(text)
    sorted_rank = {k: v for k, v in sorted(rank.items(), key=lambda item: item[1][0]['score'], reverse=True)}
    sorted_5 = {k: sorted_rank[k] for k in list(sorted_rank)[:5]}
    for j, ind in enumerate(sorted_5.keys()):
      _id = int(prediction['documents'][ind].meta['id'])

      if _id in tmp:1823
          arr[j] = 1
      else:
          arr[j] = 0

    ndcg += calc_ndcg_k(true, arr)
    i += 1

In [ ]:
ndcg_reranking = ndcg / i
ndcg_reranking = round(ndcg_reranking, 3)

print(f"Średnia NDCG wynosi {ndcg_reranking}")

Średnia NDCG wynosi 0.182


## Use a different dense encoder, e.g. E5 large or Polish Roberta Base and compute NDCG@5.

In [ ]:
from haystack.document_stores import FAISSDocumentStore

document_store_r = FAISSDocumentStore(faiss_index_factory_str="Flat", similarity="dot_product")
document_store_r.save(index_path="data_r/my_index.faiss", config_path="data_r/my_config.json")

In [ ]:
dicts = []
for id, text in zip(dataset['_id'], dataset['text']):
  dicts.append({
        'content': "</s>"+text,
        'meta': {'id': id}
    })

document_store_r.write_documents(dicts)

Writing Documents: 60000it [01:52, 532.18it/s]


In [ ]:
r_retriever = DensePassageRetriever(
    document_store=document_store_r,
    query_embedding_model= 'sdadas/mmlw-retrieval-roberta-base',
    passage_embedding_model= 'sdadas/mmlw-retrieval-roberta-base'
)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/75.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/249M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Auto-detected model language: english


In [2]:
document_store_r.update_embeddings(r_retriever)

In [ ]:
from haystack.pipelines import DocumentSearchPipeline

r_pipe = DocumentSearchPipeline(r_retriever)

In [ ]:
arr = np.array([0.0 for i in range(5)])
ndcg = 0
i = 0

for query_id in df_qa_test['query-id'].unique():
    query = str('zapytanie:' + df_queries[df_queries['_id'] == str(query_id)].iloc[0]['text'])
    prediction = r_pipe.run(
    query=query, params={"Retriever": {"top_k": 5}}
    )
    corpus_ids = df_qa_test[df_qa_test['query-id'] == query_id]['corpus-id']

    tmp = set()
    for ind in corpus_ids:
        tmp.add(int(ind))

    matches = min(max_matches.loc[query_id], 5)
    true = np.array([0.0 for i in range(5)])
    true[:matches] = 1
    for ind, val in enumerate(prediction['documents']):
        _id = int(val.meta['id'])
        if _id in tmp:
            arr[ind] = 1
        else:
            arr[ind] = 0
    ndcg += calc_ndcg_k(true, arr)
    i += 1

In [ ]:
mean_ndcg_roberta = ndcg / i
mean_ndcg_roberta = round(mean_ndcg_roberta, 3)
print(f"Średnia NDCG wynosi {mean_ndcg_roberta}")

Średnia NDCG wynosi 0.273


In [ ]:
df = pd.DataFrame({
    "silver-retriever": [mean_ndcg_silver],
    "silver-retriever_re-ranking": [ndcg_reranking],
    "roberta-base": [mean_ndcg_roberta]
})
df

,silver-retriever,silver-retriever_re-ranking,roberta-base
0,0.232,0.182,0.273


*   Which of the methods: lexical match (e.g. ElasticSearch) or dense representation works better?

W moim przypadku lepiej sprawdza się dense representation, dla którego
średnia NDCG wynosi 0.23, natomiast w przypadku ElasticSearch wynik ten nie przekraczał 0.18.

*   Which of the methods is faster?
Szybsza jest metoda z użyciem ElasticSearch
*   Try to determine the other pros and cons of using lexical search and dense document retrieval models.
  *   lexical search
      - niska tolerancja na błędy i literówki
      - szybkość działania
  *   dense representation
      - większa złożoność obliczeniowa i mniejsza interpetowalność
      - większa tolerancja na błędy